In [15]:
import sys
import numpy as np

from gamspy import (
    Container,Set,Alias,Parameter,Variable,Equation,Model,Problem,Sense,Options,
    Domain,Number,Sum,Product,Smax,Smin,Ord,Card,SpecialValues,
)
import gamspy.math as gpm

options = Options(seed=42)
m = Container(options=options)

i = Set(m,'player',records=['Harry_Potter', 'Ron_Weasley', 'Fred_Weasley', 'George_Weasley', 
    'Oliver_Wood', 'Angelina_Johnson', 'Ginny_Weasley', 'Hermione_Granger', 
    'Neville_Longbottom', 'Seamus_Finnegan', 'Dean_Thomas',
    'Romilda_Vane', 'Colin_Creevy', 'Dennis_Creevy', 'Lavender_Brown',
    'Alicia_Spinnet', 'Katie_Bell', 'Cormac_McLaggen','Demelza_Robinson'])
j = Set(m,'position')
required = Parameter(m,'required', domain=j, domain_forwarding = True,
    records=[ ('seeker', 1), ('chaser', 3), ('beater', 2), ('keeper', 1) ])

quality = Parameter(m,'quality',domain=[i,j])
quality[i,'seeker'] = gpm.uniform(32,36)
quality[i,'chaser'] = gpm.uniform(38,41)
quality[i,'beater'] = gpm.uniform(30,35)
quality[i,'keeper'] = gpm.uniform(28,38)
quality['Harry_Potter', 'seeker'] = 42

display(quality.pivot())

x = Variable(m, 'x', domain=[i, j], type='positive')

player_constraint = Equation(m, 'player_constraint', domain=i)
player_constraint[i] = Sum(j, x[i, j]) <= 1

position_constraint = Equation(m, 'position_constraint', domain=j)
position_constraint[j] = Sum(i, x[i, j]) == required[j]

objective = Sum([i, j], quality[i, j] * x[i, j])

Gryffindor = Model(m, 
              name='Gryffindor', 
              equations=m.getEquations(), 
              problem=Problem.LP, 
              sense=Sense.MAX, 
              objective=objective)

,seeker,chaser,beater,keeper
Harry_Potter,42.000000,40.107844,34.080352,30.345572
Ron_Weasley,33.423164,39.497351,31.399243,29.848148
Fred_Weasley,34.623634,39.948404,34.917313,33.049643
George_Weasley,33.381789,39.520000,32.473133,30.623237
Oliver_Wood,35.784648,40.498367,33.711638,32.795699
Angelina_Johnson,34.719680,38.767801,30.151053,31.857933
Ginny_Weasley,33.472440,39.018418,33.268533,33.185437
Hermione_Granger,34.586713,40.010411,32.978259,29.282914
Neville_Longbottom,33.434833,38.171924,31.294828,31.793913
Seamus_Finnegan,35.783428,38.085160,34.792175,30.316590


In [16]:
# put your code here
Gryffindor.solve()

,Solver Status,Model Status,Objective,Num of Equations,Num of Variables,Model Type,Solver,Solver Time
0,Normal,OptimalGlobal,270.703233538,24,77,LP,CPLEX,0.015


In [17]:
houseScore = Parameter(m,'houseScore')
houseScore[:] = Gryffindor.objective_value

team = Set(m,'Gryffindor_team',[i,j])
team[i,j] = Number(1).where[x.l[i,j] > 0.001]
print(f"Score = {houseScore.toValue()}")
display(team.pivot(fill_value=''))

Score = 270.703233538


,seeker,chaser,beater,keeper
Harry_Potter,True,,,
Fred_Weasley,,,True,
Oliver_Wood,,True,,
Dean_Thomas,,True,,
Colin_Creevy,,,,True
Cormac_McLaggen,,,True,
Demelza_Robinson,,True,,
